In [ ]:
import pandas as pd
from collections import defaultdict
from pyvi import ViTokenizer
import os
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
path = os.getcwd()
path_vsfc_train = './data/UIT-VSFC/train'
path_vsfc_test = './data/UIT-VSFC/test'
path_stopwords = os.path.join(path,'dict/vietnamese-stopwords.txt')
path_pos_sent = os.path.join(path,'dict/pos_sent.txt')
path_neg_sent = os.path.join(path,'dict/neg_sent.txt')
path_not = os.path.join(path,'dict/not.txt')

In [ ]:
data_victsd = pd.read_csv('./data/UIT-ViCTSD/UIT-ViCTSD_train.csv', encoding='utf-8').iloc[:, 1:6]
data_victsd

,Comment,Constructiveness,Toxicity,Title,Topic
0,Thật tuyệt vời...!!!,0,0,Những 'bước tiến diệu kỳ' của Trúc Nhi - Diệu Nhi,SucKhoe
1,"mỹ đã tuột dốc quá nhiều rồi, giờ muốn vực dậy...",1,0,Hình tượng Mỹ sụp đổ trong lòng người dân thế ...,TheGioi
2,tôi thấy người lái xe hơi bấm còi mới là người...,1,1,Cả trăm người đạp xe thể dục bịt kín đường,OtoXemay
3,Coi dịch là giặc. Đã mang tên đó mà xâm nhập V...,0,0,11 ngày không lây nhiễm nCoV cộng đồng,SucKhoe
4,Thương các bé quá! Các con còn quá nhỏ mà đã p...,0,0,5 trẻ chết đuối dưới ao,ThoiSu
...,...,...,...,...,...
6995,"đọc tin mà đau lòng quá,thảm nạn cứ xãy ra hoà...",0,0,5 trẻ chết đuối dưới ao,ThoiSu
6996,Nên mua bạn nhé. Để bảo vệ sức khỏe bạn trong ...,1,0,Chưa lập gia đình có nên mua bảo hiểm nhân thọ?,KinhDoanh
6997,Nên ra luật người sử dụng ma túy cũng phạm tội...,1,0,102 người dương tính ma túy trong quán bar,PhapLuat
6998,Mua Fadil đủ lăn bánh. Nếu thấy đắt thì đi Mor...,1,0,450 triệu mua xe gì phục vụ gia đình?,OtoXemay


In [ ]:
with open(path_pos_sent,'r',encoding='utf-8') as pos:
    pos_sent = pos.readlines()
pos_sent = [i.replace('\n','') for i in pos_sent]

with open(path_neg_sent,'r',encoding='utf-8') as neg:
    neg_sent = neg.readlines()[1:]
neg_sent = [i.replace('\n','') for i in neg_sent]

with open(path_not,'r',encoding='utf-8') as not_:
    not_list = not_.readlines()
not_list = [i.replace('\n','') for i in not_list]

with open(path_stopwords,'r',encoding='UTF8') as sw:
    stop_words=[i.replace('\n','') for i in sw.readlines()]

In [ ]:
replace_list ={'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ','uá':'úa','uà':'ùa','uả':'ủa','ũa':'uã','uạ':'ụa', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #emoji -> text   
        '😔': ' negative ','😡': ' negative ','⭐': ' positive ','😍': ' positive ','🙂': ' positive ',
        '😤': ' negative ','🤣': ' positive ','😡': ' negative ','🤗': ' positive ','😁': ' positive ',
        '🌼': ' positive ','😚': ' positive ','😅': ' neutral ','😞': ' negative ','😀': ' postive ',
        '👍': ' positive ','💯': ' positive ','😥': ' negative ','❤': ' positive ','😱': ' negative ',
        '😎': ' positive ','👌': ' positive ','🤬': ' negative ','😋': ' positive ','😄': ' positive ',
        '💳': ' neutral ','😖': ' negative ','😊': ' positive ',
        #Chuẩn hóa sentiment word
        ':))': ' positive ', ':)': ' positive ', ':v':' positive ','“': ' ',
        'bit':' biết ','bik':' biết ','bk':' biết ','bít':' biết ','biet':' biết ','beautiful': ' đẹp tuyệt vời','bin':' pin ','bt':' bình thường ',
        ' co ':' có ','chs ':' chơi ','choi':' chơi ','củng':' cũng ','cung':' cũng ','chất lg':' chất lượng ',
        'cx':' cũng ','cungx':' cũng ','chiệu':' chịu ','chăc':' chắc ','chac':' chắc ','chắt':' chắc ',
        'dung':' dùng ','đỗi':' đổi ','dẹp':' đẹp ','đep':' đẹp ','dây':' đây ',
        'đx':' được ','dk':' được ','dc':' được ','đc':' được ',' đ ':' được ',' đả ':' đã ','đt ':' điện thoại ','dt':' điện thoại ','đth ':' điện thoại ',
        ' e ':' em ','fb ':' facebook ','face':' facebook ',
        'gét':' ghét ','ghet':' ghét ',' gi ':' gì ','gud':' tốt ','god':' tốt ','good':' tốt ','gút':' tốt ',
        'kg ':' không ','not':' không ',' k ':' không ',' kh ':' không ',' kô ':' không ',' ko ':' không ',
        'khong':' không ','khoản ':' khoảng ','khuyến':' khuyên ','kem':' kém ',
        ' hok ':' không ','hìh':' hình ','he he':' positive ','hehe':' positive ','hihi':' positive ','haha':' positive ',
        'hjhj':' positive ','huhu':' nagative ','hàg': ' hàng ',   
        'luân':' luôn ','lun':' luôn ','luon':' luôn ',
        'lau':' lâu ','loi':' lỗi ','lổi':' lỗi ',
        'mik':' mình ','minh':' mình ','mk ':' mình ',' m ':' mình ','mak':' mà ',' ma ':' mà ','mấy':' máy ','may':' máy ','mạn ':' mạng ','mang':' mạng ',
        'mn':' mọi người ','mược':' mượt ','muoc':' mượt ','muot':' mượt ','ms':' mà ',
        'nhung':' nhưng ',' nhưn ':' nhưng ',' ng ':' người ','nv':' nhân viên ','nt':' nhắn tin ','nhak':' nha ','nhậy':' nhạy ',
        'ô kêi': ' ok ', 'okie':' ok ',' o kê ':' ok ','okey':' ok ','ôkê':' ok ','oki':' ok ','oke':' ok ','okay':' ok ','okê' :' ok ',
        'pó':' bó ','qá ':' qúa ',' pi ':' bị ','perfect':' rất tốt ',' pos ':'positive',
        ' r ':' rồi ','rùi':' rồi ','roi':' rồi ','rui':' rồi ',
        'sài':' xài ','sd':' sử dụng ','sử dụg':' sử dụng ',
        'sấu':' xấu ','sad':' tệ ',
        'tks':' cám ơn ','thks':' cám ơn ','thanks':' cám ơn ','ths':' cám ơn ','thank ':' cám ơn ','thich':' thích ','thik':' thích ',
        'tak':' ta ','tắc':' tắt ','tăc':' tắt ','tat':' tắt ',
        'thay':' thấy ','tot':' tốt ','tra':' trả ','tut':' tụt ','tuộc':' tụt ','tuột':' tụt ','tục':' tụt ',' t ':' tôi ','tui':' tôi ','thiệc':'thiệt',' tl ':'trả lời',' trl ':'trả lời',
        ' up ':' update ',' updat ':' update ',' uong ':' uổng ',' vs ':' với ',' va ':' và ',' vẩn ':' vẫn ',' van ':' vẫn ',
        'wa':' qúa ','wá':' qúa ','wua':' qúa ','wed':' web ','wf':' wifi ',' wjfj ':' wifi ',
        'xau':' xấu ','xạc':' sạc ',' yt ':' youtube ','youtu ':' youtube ',' iu ':' yêu ',' zl ':' zalo ',
        }

In [ ]:
def read_file(path, type_file='csv'):
    if type_file == 'csv':
        return pd.read_csv(path, encoding='utf-8')
    elif type_file == 'txt':
        with open(path, 'r', encoding='utf-8') as f:
            return f.readlines()

In [ ]:
from vncorenlp import VnCoreNLP
def tokenize_word(text):
    client = VnCoreNLP(address="http://127.0.0.1", port=9001)
    wordlist = client.tokenize(text)
    return ' '.join(wordlist)

In [ ]:
def clean_data(text):
    for k, v in replace_list.items():
        text = text.replace(k, v)
    translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
    text = text.translate(translator)
    text = ViTokenizer.tokenize(text)
    text = ' '.join([i for i in text.split() if len(i)>=2])
    #text = tokenize_word(text)
    def phudinh(text):
        texts =text.split()
        len_text = len(texts)
        for i in range(len_text):
            cp_text = texts[i]
            if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
                numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1
                for j in range(numb_word):
                    if texts[i + j + 1] in pos_sent:
                        texts[i] = 'notpos'
                        texts[i + j + 1] = ''
                    if texts[i + j + 1] in neg_sent:
                        texts[i] = 'notneg'
                        texts[i + j + 1] = ''
            else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp positive)
                if cp_text in pos_sent:
                    texts.append('positive')
                elif cp_text in neg_sent:
                    texts.append('negative')
        return ' '.join(texts)
    text = phudinh(text)
    return text

In [ ]:
def txt2dataframe(path):
    dataset = pd.DataFrame(columns=['sentiments', 'sents', 'topics'])
    for name, file in zip(dataset.columns, os.listdir(path)):
        data = read_file(os.path.join(path, file), 'txt')
        dataset[name] = [text.replace('\n', '') for text in data]
    return dataset

In [ ]:
data_train = txt2dataframe(path_vsfc_train)
data_test = txt2dataframe(path_vsfc_test)
data_train['sents'].apply(clean_data)
data_test['sents'].apply(clean_data)

0                                  nói tiếng anh lưu_loát
1                                  giáo_viên rất vui_tính
2                                           cô max có tâm
3                                giảng bài thu_hút dí_dỏm
4       giáo_viên không giảng_dạy kiến_thức hướng_dẫn ...
                              ...                        
3161    các slide khó_hiểu ngôn_ngữ trong slide phức_t...
3162                     giáo_viên giảng_dạy có tâm_huyết
3163                        chia_sẻ cho em nhiều điều hay
3164                            em tiếp_thu chậm negative
3165    em có_học một trung_tâm tiếng anh ngoài trường...
Name: sents, Length: 3166, dtype: object

In [ ]:
data_train

,sentiments,sents,topics
0,2,slide giáo trình đầy đủ .,1
1,2,"nhiệt tình giảng dạy , gần gũi với sinh viên .",0
2,0,đi học đầy đủ full điểm chuyên cần .,1
3,0,chưa áp dụng công nghệ thông tin và các thiết ...,0
4,2,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",0
...,...,...,...
11421,0,chỉ vì môn game mà em học hai lần mà không qua...,1
11422,2,em cảm ơn cô nhiều .,0
11423,0,giao bài tập quá nhiều .,0
11424,2,"giáo viên dạy dễ hiểu , nhiệt tình .",0


In [ ]:
X_train = data_train.iloc[:, 1]
Y_train = data_train.iloc[:, -1]
X_test = data_test.iloc[:, 1]
Y_test = data_test.iloc[:, -1]

In [ ]:
X_train

0                                slide giáo trình đầy đủ .
1           nhiệt tình giảng dạy , gần gũi với sinh viên .
2                     đi học đầy đủ full điểm chuyên cần .
3        chưa áp dụng công nghệ thông tin và các thiết ...
4        thầy giảng bài hay , có nhiều bài tập ví dụ ng...
                               ...                        
11421    chỉ vì môn game mà em học hai lần mà không qua...
11422                                 em cảm ơn cô nhiều .
11423                             giao bài tập quá nhiều .
11424                 giáo viên dạy dễ hiểu , nhiệt tình .
11425    gói gọn doubledot hay , tận tình , phù hợp với...
Name: sents, Length: 11426, dtype: object

In [ ]:
from sklearn import naive_bayes, svm
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
steps=[]
steps.append(('CountVectorizer', CountVectorizer(ngram_range=(1,3),max_df=0.5, min_df=5)))
steps.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
steps.append(('classifier', svm.SVC(probability=True, kernel="poly", class_weight="balanced")))
clf = Pipeline(steps)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)

In [ ]:
clf.score(X_test,Y_test)

0.8373341756159192

In [ ]:
steps=[]
steps.append(('CountVectorizer', CountVectorizer(ngram_range=(1,3),max_df=0.5, min_df=5)))
steps.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
steps.append(('classifier', naive_bayes.MultinomialNB()))
naive = Pipeline(steps)
naive.fit(X_train, Y_train)
y_pred = naive.predict(X_test)

In [ ]:
naive.score(X_test,Y_test)

0.8316487681617183